# Social Data Preprocessing

In [16]:
### package
import os 
import pandas as pd
import numpy as np
import copy
os.chdir("C:/Users/user/Desktop/Social/2021")

### 데이터 로딩

In [17]:
### ESG
raw_esg = pd.read_excel("ESG_DATA.xlsx",converters={'종목코드':str})

### DART
raw_dart20 = pd.read_excel("DART직원_2020.xlsx",converters={'종목코드':str})
raw_dart21 = pd.read_excel("DART직원_2021.xlsx",converters={'종목코드':str})
raw_chief = pd.read_excel("DART임원_2021.xlsx",converters={'종목코드':str})

### 가족친화인증
raw_fam = pd.read_excel("가족친화인증기업_2021.xlsx",converters={'종목코드':str})

### ESG채권
raw_bond1 = pd.read_excel("국내사회적채권_2021.xlsx")
raw_bond2 = pd.read_excel("국외사회적채권_2021.xlsx") 

### 고용평등우수 & 일자리으뜸 & 인적자원인증 
raw_eq1 = pd.read_excel("고용평등우수_2021.xlsx")
raw_eq2 = pd.read_excel("인적자원인증_2021.xlsx")
raw_eq3 = pd.read_excel("일자리으뜸_2021.xlsx")

### 지속가능경영보고서
raw_report1 = pd.read_excel("DART지가경공시_2021.xlsx")
raw_report2 = pd.read_excel("KSA지가경_2021.xlsx")

### 공정위위반사항
raw_ill = pd.read_excel("공정위위반사항_2021.xlsx")

### ESG임원관련(금융회사지배구조)
raw_fgover = pd.read_excel("금융지배구조_2021.xlsx")

### 소비자중심경영인증기업
raw_cust = pd.read_excel("소비자중심경영인증기업_2021.xlsx",converters={'종목코드':str})

### 지역사회공헌인증
raw_soc = pd.read_excel("지역사회공헌인증_2021.xlsx",converters={'종목코드':str})

### 정보보호
raw_sec1 = pd.read_excel("정보보호_2021.xlsx",converters={'종목코드':str})

### 정보보호 데이터
raw_sec2 = pd.read_excel("정보보호데이터_2021.xlsx",converters={'종목코드':str})

### 잡플래닛평점
raw_job = pd.read_excel("잡플래닛평점_2021.xlsx",converters={'종목코드':str})

### 지속가능경영보고서 사회부문
raw_social_report = pd.read_excel("사회_2021.xlsx")

## ISMS-P 인증
raw_isms = pd.read_excel("ISMS-P_2021.xlsx",converters={'종목코드':str})

### 분석대상 key id

In [18]:
### 분석대상 key 변수 : 분석대상에서 SBS미디어홀딩스, 에스케이머티리얼즈 제외
raw_esg = raw_esg[-raw_esg['종목코드'].isin(['101060','036490'])]
code = list(raw_esg['종목코드'])
corp = list(raw_esg['회사명'])
idx_esg = raw_esg[['종목코드','회사명']]
sector_esg = raw_esg[['종목코드','회사명','SICS_Sector_Kr']]

### Social 데이터 분석대상 Filtering

In [19]:
### DART
dart20 = raw_dart20[raw_dart20['종목코드'].isin(code)]
dart21 = raw_dart21[raw_dart21['종목코드'].isin(code)]
chief21 = raw_chief[raw_chief['종목코드'].isin(code)]

### 가족친화
fam = raw_fam[raw_fam['종목코드'].isin(code)]

### ESG채권
bond1 = pd.merge(left = raw_bond1, right = idx_esg, how = 'left', on = '회사명')
bond2 = pd.merge(left = raw_bond2, right = idx_esg, how = 'left', on = '회사명')

### 고용평등우수 & 일자리으뜸 & 인적자원인증
eq1 = pd.merge(left = raw_eq1, right = idx_esg, how = 'left', on = '회사명')
eq2 = pd.merge(left = raw_eq2, right = idx_esg, how = 'left', on = '회사명')
eq3 = pd.merge(left = raw_eq3, right = idx_esg, how = 'left', on = '회사명')

### 지속가능경영보고서
report1 = pd.merge(left = raw_report1, right = idx_esg, how = 'left', on = '회사명')
report2 = pd.merge(left = raw_report2, right = idx_esg, how = 'inner', on = '회사명')

### 공정위위반사항
ill = pd.merge(left = raw_ill, right = idx_esg, how = 'left', on = '회사명')

### ESG임원 관련 (금융회사지배구조보고서)
fgover = pd.merge(left = raw_fgover, right = idx_esg, how = 'left', on = '회사명')

### 지속가능경영보고서 사회
social_report = pd.merge(left = raw_social_report, right = idx_esg, how = 'left', on = '회사명')

### Data Preprocessing

In [20]:
### DART

## 임원
r_chief21 = chief21[-chief21['등기임원여부'].isin(['미등기임원','미등기'])]

# 총임원
total_chief = r_chief21.groupby('종목코드').size().reset_index().rename(columns = {0:'총이사'})

# 여성임원
woman_chief = r_chief21[r_chief21['성별'] == '여'].groupby('종목코드').size().reset_index().rename(columns = {0:'여성이사'})

# ESG관련 이사 수 : ESG, 지속가능, 미래전략, 지속경영 키워드가 관련된 ESG이사 추출
chief_esg = list(r_chief21['담당업무'])
for i in range(r_chief21.shape[0]) :
    if any(word in chief_esg[i] for word in ['ESG','esg','지속가능','미래전략','지속경영']) :
        chief_esg[i] = 1
    else : chief_esg[i] = 0
        
c = pd.DataFrame({'종목코드':list(r_chief21['종목코드']),
                  'ESG이사':chief_esg})
esg_chief = c.groupby('종목코드')['ESG이사'].sum().reset_index()

# 임원 데이터 merge
chief21_1 = pd.merge(left = total_chief, right = woman_chief, on='종목코드', how='left')
new_chief21 = pd.merge(left = chief21_1, right = esg_chief, on='종목코드', how='left')
new_chief21['여성임원비율'] = new_chief21['여성이사']/new_chief21['총이사']
new_chief21['ESG임원비율'] = new_chief21['ESG이사']/new_chief21['총이사']
r_dart3 = pd.concat([fgover[['종목코드','ESG임원비율']],new_chief21[['종목코드','ESG임원비율']]]).drop_duplicates("종목코드", keep = 'first')
new_chief21 = pd.merge(left = new_chief21.drop(['ESG임원비율'], axis = 1), right = r_dart3, on = '종목코드', how = 'left')

new_chief21.drop(['총이사','여성이사','ESG이사'], axis=1, inplace = True)
new_chief21 = new_chief21.fillna(0)

## 여성미등기임원
w_chief21 = chief21[chief21['등기임원여부'].isin(['미등기임원','미등기'])]
wchief21 = w_chief21[w_chief21['성별'] == '여'].groupby(['종목코드']).count().reset_index()[['종목코드','회사명']].rename(columns = {'회사명':'여성임원수'})
tchief21 = w_chief21.groupby('종목코드').count().reset_index()[['종목코드','회사명']].rename(columns = {'회사명':'임원수'})

new_Wchief21 = pd.merge(left = tchief21, right = wchief21, how = 'left', on = '종목코드')
new_Wchief21['미등기여성임원비율21'] = new_Wchief21['여성임원수']/new_Wchief21['임원수']
new_Wchief21 = new_Wchief21.fillna(0)
new_Wchief21.drop(['임원수','여성임원수'],axis=1,inplace = True)

In [21]:
### 근속년수 data 전처리

#기본 패키지들
import pyperclip
from numpy import nan as NA

#정규표현식 패키지
import re
from itertools import groupby

bracket = "\(.*\)" #정규표현식의 괄호 정의

# 문자열변환
def trans_str(string):
    # STEP 1 -------------------------------------------------
    # string에 있는 약, 공백을 전부 제거
    string = str(string).replace('약', '').replace(' ', '').replace('\n', '')
    # re.sub으로 string 안의 괄호를 ''으로 치환 
    string = re.sub(bracket, '', string)
    # STEP 2 -------------------------------------------------
    string = string.replace('년', 'Y')
    string = string.replace('개월', "M").replace('월', 'M').replace('일', 'D')
    return(string)
new_date1 = [trans_str(x) for x in dart20['평균근속연수']]
new_date2 = [trans_str(x) for x in dart21['평균근속연수']]

eng = re.compile('[a-zA-Z]+')

# function
def trans_year(string):
    # 영어 포함 X, 숫자만 있으면 pass.
    if re.search(eng, string) == None:
        return(string)
    # 영어 포함하는 경우, 영어 기준으로 문자열 split
    else: str_list = [''.join(g) for _, g in groupby(string, str.isalpha)]
    # split한 list의 뒤부터 하나씩 검토.
    try: 
        ## 만약 마지막 문자열이 D인 경우...
        if str_list[-1]=='D':
            del str_list[-1] #'D' 삭제
            day = str_list.pop(-1) #D 앞의 숫자 day에 삽입
        if str_list[-1]=='M':
            del str_list[-1] #'M' 삭제
            month = str_list.pop(-1) #M 앞의 숫자를 month에 삽입
            month = float(month)
        if str_list[-1]=='Y':
            del str_list[-1] #Y 삭제
            year = str_list.pop(-1) #year에 숫자 삽입
            year = float(year)
        # 만약 year, month, day 객체가 존재하지 않을 경우엔 각 객체에 0 삽입
        try:
            year
        except NameError:
            year=0
        try:
            month
        except NameError:
            month=0
        try:
            day
        except NameError:
            day=0

        return(round((year + month/12 + int(day)/365), 2))
            
    except IndexError:
        try:
            year
        except NameError:
            year=0
        try:
            month
        except NameError:
            month=0
        try:
            day
        except NameError:
            day=0

        return(round((year + month/12 + int(day)/365), 2))

result1 = [trans_year(date) for date in new_date1]
result2 = [trans_year(date) for date in new_date2]

dart20['new_근속연수'] = result1
dart21['new_근속연수'] = result2

C:\Users\user\AppData\Local\Temp\ipykernel_7000\3116069615.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dart20['new_근속연수'] = result1
C:\Users\user\AppData\Local\Temp\ipykernel_7000\3116069615.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dart21['new_근속연수'] = result2


In [22]:
### 2020년 DART 직원
r_dart20 = pd.concat([dart20.iloc[:,:6],dart20.iloc[:,6:].replace("-",0)],axis = 1)
r_dart20['new_근속연수'] = list(map(float,r_dart20['new_근속연수']))
to_dart20 = r_dart20[r_dart20['사업부문'].str.contains('합계')|r_dart20['사업부문'].str.contains('총계')].reset_index()
rm_dart20 = r_dart20[-r_dart20['종목코드'].isin(to_dart20['종목코드'])].reset_index()

## 합계로 되어있는 기업

# 총직원수
to_emp_total = to_dart20.groupby(['종목코드'])['합계'].sum().reset_index()
# 여성직원수
to_emp_woman = to_dart20[to_dart20['성별'] == '여'].groupby(['종목코드'])['합계'].sum().reset_index().rename(columns = {'합계':'여성직원수'})
# 기간제직원수
to_emp_temp = to_dart20.groupby(['종목코드'])['기간제 근로자(전체)'].sum().reset_index().rename(columns ={'기간제 근로자(전체)':'기간제'})
# 여성 평균 금액
to_emp_Wpay = to_dart20[to_dart20['성별'] == '여'][['종목코드','1인평균급여액']].rename(columns = {'1인평균급여액':'여성평균급여액'})
# 남성 평균 금액
to_emp_Mpay = to_dart20[to_dart20['성별'] == '남'][['종목코드','1인평균급여액']].rename(columns = {'1인평균급여액':'남성평균급여액'})
# 남성 평균 근속연수
to_emp_Mwork = to_dart20[to_dart20['성별'] == '남'][['종목코드','new_근속연수']].rename(columns = {'new_근속연수':'남성평균근속연수'})
# 여성 평균 근속연수
to_emp_Wwork = to_dart20[to_dart20['성별'] == '여'][['종목코드','new_근속연수']].rename(columns = {'new_근속연수':'여성평균근속연수'})

## 개별적으로 있는 기업

# 급여
rm1 = rm_dart20.groupby(["종목코드","성별"])[["합계","기간제 근로자(전체)","연간급여총액"]].sum().reset_index()
rm1['1인평균급여액'] = rm1['연간급여총액']/rm1['합계']

# 근속연수
rm2 = rm_dart20[['종목코드','성별','합계','new_근속연수']]
rm2['근속연수sum'] = rm2['합계'] * rm2['new_근속연수']
rm3 = rm2.groupby(['종목코드','성별'])['합계','근속연수sum'].sum().reset_index()
rm3['평균근속연수'] = rm3['근속연수sum'] / rm3['합계']

# 총 직원수
rm_emp_total = rm1.groupby("종목코드")["합계"].sum().reset_index()
# 여성직원수
rm_emp_woman = rm1[rm1["성별"] == "여"][['종목코드','합계']].rename(columns = {'합계':'여성직원수'})
# 기간제직원수
rm_emp_temp = rm1.groupby("종목코드")["기간제 근로자(전체)"].sum().reset_index().rename(columns ={'기간제 근로자(전체)':'기간제'})
# 여성 평균 금액
rm_emp_Wpay = rm1[rm1["성별"] == "여"][['종목코드','1인평균급여액']].rename(columns = {'1인평균급여액':'여성평균급여액'})
# 남성 평균 금액
rm_emp_Mpay = rm1[rm1["성별"] == "남"][['종목코드','1인평균급여액']].rename(columns = {'1인평균급여액':'남성평균급여액'})
# 남성 평균 근속연수
rm_emp_Mwork = rm3[rm3['성별'] == '남'][['종목코드','평균근속연수']].rename(columns = {'평균근속연수':'남성평균근속연수'})
# 여성 평균 근속연수
rm_emp_Wwork = rm3[rm3['성별'] == '여'][['종목코드','평균근속연수']].rename(columns = {'평균근속연수':'여성평균근속연수'})

## merge
to_emp = pd.merge(left = to_emp_total, right = to_emp_woman, how = 'left', on = '종목코드')
to_emp = pd.merge(left = to_emp, right = to_emp_temp, how = 'left', on = '종목코드')
to_emp = pd.merge(left = to_emp, right = to_emp_Mpay, how = 'left', on = '종목코드')
to_emp = pd.merge(left = to_emp, right = to_emp_Wpay, how = 'left', on = '종목코드')
to_emp = pd.merge(left = to_emp, right = to_emp_Mwork, how = 'left', on = '종목코드')
to_emp = pd.merge(left = to_emp, right = to_emp_Wwork, how = 'left', on = '종목코드')

rm_emp = pd.merge(left = rm_emp_total, right = rm_emp_woman, how = 'left', on = '종목코드')
rm_emp = pd.merge(left = rm_emp, right = rm_emp_temp, how = 'left', on = '종목코드')
rm_emp = pd.merge(left = rm_emp, right = rm_emp_Mpay, how = 'left', on = '종목코드')
rm_emp = pd.merge(left = rm_emp, right = rm_emp_Wpay, how = 'left', on = '종목코드')
rm_emp = pd.merge(left = rm_emp, right = rm_emp_Mwork, how = 'left', on = '종목코드')
rm_emp = pd.merge(left = rm_emp, right = rm_emp_Wwork, how = 'left', on = '종목코드')

emp20 = pd.concat([to_emp, rm_emp])

emp20["여성직원비율20"] = emp20['여성직원수']/emp20['합계']
emp20["기간제비율20"] = emp20["기간제"]/emp20["합계"]
emp20["여성평균급여대비총합평균20"] = emp20["여성평균급여액"]/(emp20["여성평균급여액"]+emp20["남성평균급여액"])
emp20["여성평균근속연수대비총합평균20"] = emp20["여성평균근속연수"]/(emp20["여성평균근속연수"]+emp20["남성평균근속연수"])

emp20.drop(["여성직원수","기간제","남성평균급여액","여성평균급여액","남성평균근속연수","여성평균근속연수"],axis=1,inplace= True)
emp20 = emp20.rename(columns = {'합계':'합계20'})

C:\Users\user\AppData\Local\Temp\ipykernel_7000\3786437162.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rm2['근속연수sum'] = rm2['합계'] * rm2['new_근속연수']
C:\Users\user\AppData\Local\Temp\ipykernel_7000\3786437162.py:33: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  rm3 = rm2.groupby(['종목코드','성별'])['합계','근속연수sum'].sum().reset_index()


In [23]:
### 2021년 DART 직원
r_dart21 = pd.concat([dart21.iloc[:,:6],dart21.iloc[:,6:].replace("-",0)],axis = 1)
r_dart21['new_근속연수'] = list(map(float,r_dart21['new_근속연수']))
to_dart21 = r_dart21[r_dart21['사업부문'].str.contains('합계')|r_dart21['사업부문'].str.contains('총계')].reset_index()
rm_dart21 = r_dart21[-r_dart21['종목코드'].isin(to_dart21['종목코드'])].reset_index()

## 합계로 되어있는 기업

# 총직원수
to_emp_total = to_dart21.groupby(['종목코드'])['합계'].sum().reset_index()
# 여성직원수
to_emp_woman = to_dart21[to_dart21['성별'] == '여'].groupby(['종목코드'])['합계'].sum().reset_index().rename(columns = {'합계':'여성직원수'})
# 기간제직원수
to_emp_temp = to_dart21.groupby(['종목코드'])['기간제 근로자(전체)'].sum().reset_index().rename(columns ={'기간제 근로자(전체)':'기간제'})
# 여성 평균 금액
to_emp_Wpay = to_dart21[to_dart21['성별'] == '여'][['종목코드','1인평균급여액']].rename(columns = {'1인평균급여액':'여성평균급여액'})
# 남성 평균 금액
to_emp_Mpay = to_dart21[to_dart21['성별'] == '남'][['종목코드','1인평균급여액']].rename(columns = {'1인평균급여액':'남성평균급여액'})
# 남성 평균 근속연수
to_emp_Mwork = to_dart21[to_dart21['성별'] == '남'][['종목코드','new_근속연수']].rename(columns = {'new_근속연수':'남성평균근속연수'})
# 여성 평균 근속연수
to_emp_Wwork = to_dart21[to_dart21['성별'] == '여'][['종목코드','new_근속연수']].rename(columns = {'new_근속연수':'여성평균근속연수'})

## 개별적으로 있는 기업

# 급여
rm1 = rm_dart21.groupby(["종목코드","성별"])[["합계","기간제 근로자(전체)","연간급여총액"]].sum().reset_index()
rm1['1인평균급여액'] = rm1['연간급여총액']/rm1['합계']

# 근속연수
rm2 = rm_dart21[['종목코드','성별','합계','new_근속연수']]
rm2['근속연수sum'] = rm2['합계'] * rm2['new_근속연수']
rm3 = rm2.groupby(['종목코드','성별'])['합계','근속연수sum'].sum().reset_index()
rm3['평균근속연수'] = rm3['근속연수sum'] / rm3['합계']

# 총 직원수
rm_emp_total = rm1.groupby("종목코드")["합계"].sum().reset_index()
# 여성직원수
rm_emp_woman = rm1[rm1["성별"] == "여"][['종목코드','합계']].rename(columns = {'합계':'여성직원수'})
# 기간제직원수
rm_emp_temp = rm1.groupby("종목코드")["기간제 근로자(전체)"].sum().reset_index().rename(columns ={'기간제 근로자(전체)':'기간제'})
# 여성 평균 금액
rm_emp_Wpay = rm1[rm1["성별"] == "여"][['종목코드','1인평균급여액']].rename(columns = {'1인평균급여액':'여성평균급여액'})
# 남성 평균 금액
rm_emp_Mpay = rm1[rm1["성별"] == "남"][['종목코드','1인평균급여액']].rename(columns = {'1인평균급여액':'남성평균급여액'})
# 남성 평균 근속연수
rm_emp_Mwork = rm3[rm3['성별'] == '남'][['종목코드','평균근속연수']].rename(columns = {'평균근속연수':'남성평균근속연수'})
# 여성 평균 근속연수
rm_emp_Wwork = rm3[rm3['성별'] == '여'][['종목코드','평균근속연수']].rename(columns = {'평균근속연수':'여성평균근속연수'})

## merge
to_emp = pd.merge(left = to_emp_total, right = to_emp_woman, how = 'left', on = '종목코드')
to_emp = pd.merge(left = to_emp, right = to_emp_temp, how = 'left', on = '종목코드')
to_emp = pd.merge(left = to_emp, right = to_emp_Mpay, how = 'left', on = '종목코드')
to_emp = pd.merge(left = to_emp, right = to_emp_Wpay, how = 'left', on = '종목코드')
to_emp = pd.merge(left = to_emp, right = to_emp_Mwork, how = 'left', on = '종목코드')
to_emp = pd.merge(left = to_emp, right = to_emp_Wwork, how = 'left', on = '종목코드')

rm_emp = pd.merge(left = rm_emp_total, right = rm_emp_woman, how = 'left', on = '종목코드')
rm_emp = pd.merge(left = rm_emp, right = rm_emp_temp, how = 'left', on = '종목코드')
rm_emp = pd.merge(left = rm_emp, right = rm_emp_Mpay, how = 'left', on = '종목코드')
rm_emp = pd.merge(left = rm_emp, right = rm_emp_Wpay, how = 'left', on = '종목코드')
rm_emp = pd.merge(left = rm_emp, right = rm_emp_Mwork, how = 'left', on = '종목코드')
rm_emp = pd.merge(left = rm_emp, right = rm_emp_Wwork, how = 'left', on = '종목코드')

emp21 = pd.concat([to_emp, rm_emp])

emp21["여성직원비율21"] = emp21['여성직원수']/emp21['합계']
emp21["기간제비율21"] = emp21["기간제"]/emp21["합계"]
emp21["여성평균급여대비총합평균21"] = emp21["여성평균급여액"]/(emp21["여성평균급여액"]+emp21["남성평균급여액"])
emp21["여성평균근속연수대비총합평균21"] = emp21["여성평균근속연수"]/(emp21["여성평균근속연수"]+emp21["남성평균근속연수"])

emp21.drop(["여성직원수","기간제","남성평균급여액","여성평균급여액","남성평균근속연수","여성평균근속연수"],axis=1,inplace= True)
emp21 = emp21.rename(columns = {'합계':'합계21'})

C:\Users\user\AppData\Local\Temp\ipykernel_7000\1633735007.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rm2['근속연수sum'] = rm2['합계'] * rm2['new_근속연수']
C:\Users\user\AppData\Local\Temp\ipykernel_7000\1633735007.py:33: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  rm3 = rm2.groupby(['종목코드','성별'])['합계','근속연수sum'].sum().reset_index()


In [24]:
## DART 직원 데이터 merging
new_emp = pd.merge(left = emp21, right = emp20, how = 'left', on = '종목코드')
new_emp = pd.merge(left = sector_esg[['종목코드','SICS_Sector_Kr']], right = new_emp, how = 'left', on = '종목코드')
new_emp['업종별여성비율'] = new_emp.groupby("SICS_Sector_Kr")['여성직원비율21'].apply(lambda x : x/np.mean(x))
new_emp['인력유지율'] = new_emp['합계21']/new_emp['합계20']
new_emp['정규직비율'] = 1 - new_emp['기간제비율21']

new_EMP = new_emp[['종목코드','여성직원비율21','업종별여성비율','정규직비율','인력유지율','여성평균급여대비총합평균21','여성평균근속연수대비총합평균21']]
new_EMP = new_EMP.rename(columns = {'여성직원비율21':'여성직원비율',
                                    '여성평균급여대비총합평균21':'여성평균급여대비총합',
                                    '여성평균근속연수대비총합평균21':'여성평균근속연수대비총합'})

In [25]:
### 가족친화
fam['가족친화인증'] = 1
new_fam = fam[['종목코드','가족친화인증']]
new_fam = new_fam.drop_duplicates("종목코드",keep='first')

### 고용평등우수 & 일자리으뜸 & 인적자원인증
eq = pd.concat([eq1['종목코드'],eq2['종목코드'],eq3['종목코드']]).reset_index()
eq['고용관련인증'] = 1
new_eq = eq[['종목코드','고용관련인증']]

### 지속가능경영보고서
report1['지가경공시'] = 1
report2['지가경발간'] = 1
report = pd.merge(left = report2[['종목코드','지가경발간']], 
                  right = report1[['종목코드','지가경공시']], how = 'left', on = '종목코드')
new_report = report.fillna(0).drop_duplicates('종목코드',keep='first')

### 공정위위반사항
new_ill = ill[['종목코드','합계']].rename(columns = {'합계':'공정위위반횟수'})
new_ill = new_ill.groupby("종목코드")['공정위위반횟수'].sum().reset_index()

### ESG채권
bond = pd.concat([bond1[bond1['채권종류'] != "녹색채권"]['종목코드'],bond2[bond2['채권유형'] != "그린본드"]['종목코드']]).reset_index()
bond['ESG채권'] = 1
new_bond = bond.drop_duplicates("종목코드",keep='first')[['종목코드','ESG채권']]

### 소비자중심경영인증
cust = raw_cust.drop_duplicates("종목코드", keep = 'first')
cust['소비자중심경영'] = 1
new_cust = cust[['종목코드','소비자중심경영']]

### 지역사회공헌인증
soc = raw_soc.drop_duplicates("종목코드", keep = 'first')
soc['지역사회공헌'] = 1
new_soc = soc[['종목코드','지역사회공헌']]

### 정보보호공시
sec1 = raw_sec1.drop_duplicates("종목코드", keep = 'first')
sec1['정보보호공시'] = 1
new_sec1 = sec1[['종목코드','정보보호공시']]

### 잡플래닛 평점 : 결측치는 train set에서의 평균값으로 대체
new_job = raw_job[['종목코드','score']].rename(columns = {'score':'잡플래닛평점'})
new_job = pd.merge(left = sector_esg, right = new_job, how = 'left', on ='종목코드')

## train, test split
train_idx = raw_esg.iloc[:661,1]
train_mean = new_job[new_job['종목코드'].isin(train_idx)].groupby('SICS_Sector_Kr')['잡플래닛평점'].mean().to_dict()
new_job['잡플래닛평점'] = new_job['잡플래닛평점'].fillna(pd.Series([train_mean[x] for x in new_job['SICS_Sector_Kr']]))
new_job = new_job[['종목코드','잡플래닛평점']]

### 신규채용률 : 결측치는 train set에서의 최소값으로 대체
new_social1 = social_report[['종목코드','신규채용수(2020)','신규채용수(2021)']]
new_social1['신규채용률'] = new_social1['신규채용수(2021)']/new_social1['신규채용수(2020)']
new_social1 = new_social1.drop(['신규채용수(2020)','신규채용수(2021)'],axis=1)
new_social1 = pd.merge(left = sector_esg[['종목코드','SICS_Sector_Kr']], right = new_social1, how = 'left', on = '종목코드')

## train, test split
train_idx = raw_esg.iloc[:661,1]
train_min = new_social1[new_social1['종목코드'].isin(train_idx)].groupby('SICS_Sector_Kr')['신규채용률'].min().to_dict()
new_social1['신규채용률'] = new_social1['신규채용률'].fillna(pd.Series([train_min[x] for x in new_social1['SICS_Sector_Kr']]))
new_social1 = new_social1[['종목코드','신규채용률']]

### 사회기부 및 투자
new_social2 = social_report[social_report['기부 및 투자금 (백만)'] > 0]
new_social2['기부_투자'] = 1
new_social2 = new_social2[['종목코드','기부_투자']]

C:\Users\user\AppData\Local\Temp\ipykernel_7000\1956197370.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fam['가족친화인증'] = 1
C:\Users\user\AppData\Local\Temp\ipykernel_7000\1956197370.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soc['지역사회공헌'] = 1
C:\Users\user\AppData\Local\Temp\ipykernel_7000\1956197370.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

### Social Data Merging

In [26]:
esg = raw_esg[['회사명','종목코드','SICS_Sector_Kr','S_GRADE']]
social = pd.merge(left = esg, right = new_chief21, on = '종목코드')
social = pd.merge(left = social, right = new_Wchief21, how = 'left', on = '종목코드')
social = pd.merge(left = social, right = new_EMP, how = 'left', on = '종목코드')
social = pd.merge(left = social, right = new_fam, how = 'left', on = '종목코드')
social = pd.merge(left = social, right = new_eq, how = 'left', on = '종목코드')
social = pd.merge(left = social, right = new_report, how = 'left', on = '종목코드')
social = pd.merge(left = social, right = new_ill, how = 'left', on = '종목코드')
social = pd.merge(left = social, right = new_bond, how = 'left', on = '종목코드')
social = pd.merge(left = social, right = new_cust, how = 'left', on = '종목코드')
social = pd.merge(left = social, right = new_soc, how = 'left', on = '종목코드')
social = pd.merge(left = social, right = new_sec1, how = 'left', on = '종목코드')
social = pd.merge(left = social, right = new_job, how = 'left', on = '종목코드')
social = pd.merge(left = social, right = new_social1, how = 'left', on = '종목코드')
social = pd.merge(left = social, right = new_social2, how = 'left', on = '종목코드')
social = social.fillna(0)

### Social Key Risk

In [27]:
### social key risk
skr_base = raw_esg[['종목코드','SICS_Sector_Kr']]

## 지속가능경영보고서 
new_report['지속가능경영보고서'] = (new_report['지가경공시'] + new_report['지가경발간'])/2
skr_k1 = new_report[['종목코드','지속가능경영보고서']]

## ESG채권
skr_k2 = copy.deepcopy(new_bond)

## ESG임원비율
skr_k3 = new_chief21[['종목코드','ESG임원비율']]

## 인권경영
skr_k4 = social_report[['종목코드','인권경영']]

## 사회 기부금
skr_k5 = social_report[social_report['기부 및 투자금 (백만)'] != 0]
skr_k5['사회기부/투자'] = 1
skr_k5 = skr_k5[['종목코드','사회기부/투자']]

## 지역사회공헌기업
skr_k6 = new_soc

## 정보관련공시
skr_k7 = new_sec1

## 정보관련투자 : 결측치는 train set에서의 최소값으로 대체
skr_k8 = pd.merge(left = sector_esg, right = raw_sec2[['종목코드','정보기술부문 투자액(A)','정보보호부문 투자액(B)']], how = 'left', on = '종목코드')
skr_k8.loc[skr_k8['정보보호부문 투자액(B)'] == -1,'정보보호부문 투자액(B)'] = 0
skr_k8['정보보호투자비율'] = skr_k8['정보보호부문 투자액(B)']/skr_k8['정보기술부문 투자액(A)']

# train, test split
train_idx = raw_esg.iloc[:661,1]
train_min = skr_k8[skr_k8['종목코드'].isin(train_idx)].groupby('SICS_Sector_Kr')['정보보호투자비율'].min().to_dict()
skr_k8['정보보호투자비율'] = skr_k8['정보보호투자비율'].fillna(pd.Series([train_min[x] for x in skr_k8['SICS_Sector_Kr']]))
skr_k8 = skr_k8[['종목코드','정보보호투자비율']]

## ISMS-P인증기업
skr_k9 = raw_isms.drop_duplicates("종목코드",keep = 'first')
skr_k9['ISMS-P인증'] = 1
skr_k9 = skr_k9[['종목코드','ISMS-P인증']]

## ISO9001인증
skr_k10 = social_report[['종목코드','ISO9001']]

## 노조 및 노사
skr_k11 = social_report[['종목코드','노조 및 노사']]

## 정규직비율
skr_k12 = new_EMP[['종목코드','정규직비율']]

## 산업재해율 (1-산재율) : 결측치는 train set에서의 최소값으로 대체
skr_k13 = social_report[['종목코드','산업재해율']]
skr_k13['1-산재율'] = (100 - skr_k13['산업재해율'])/100
skr_k13 = skr_k13[['종목코드','1-산재율']]
skr_k13 = pd.merge(left = sector_esg[['종목코드','SICS_Sector_Kr']], right = skr_k13, how = 'left', on = '종목코드')

## train, test split
train_idx = raw_esg.iloc[:661,1]
train_min = skr_k13[skr_k13['종목코드'].isin(train_idx)].groupby('SICS_Sector_Kr')['1-산재율'].min().to_dict()
skr_k13['산재율점수'] = skr_k13['1-산재율'].fillna(pd.Series([train_min[x] for x in skr_k13['SICS_Sector_Kr']]))
skr_k13 = skr_k13[['종목코드','산재율점수']]

## 안전교육
skr_k14 = social_report[['종목코드','안전교육']]

## ISO45001인증
skr_k15 = social_report[['종목코드','ISO45001']]

## 종업원 다양성
skr_k16 = pd.merge(left = new_EMP[['종목코드','여성직원비율','여성평균급여대비총합','여성평균근속연수대비총합']], right = new_Wchief21, how = 'left', on = '종목코드')
skr_k16 = skr_k16.fillna(0)
skr_k16['여성평균급여대비총합'] = 1 - abs(skr_k16['여성평균급여대비총합'] - 0.5)
skr_k16['여성평균근속연수대비총합'] = 1 - abs(skr_k16['여성평균근속연수대비총합'] - 0.5)
skr_k16['여성직원'] = 1 - abs((skr_k16['여성직원비율'] + skr_k16['미등기여성임원비율21'])/2 - 0.5)
skr_k16 = skr_k16.iloc[:,[0,2,3,5]]

## social key risk data set merge 
skr = pd.merge(left = skr_base, right = skr_k1, how = 'left', on = '종목코드')
skr = pd.merge(left = skr, right = skr_k2, how = 'left', on = '종목코드')
skr = pd.merge(left = skr, right = skr_k3, how = 'left', on = '종목코드')
skr = pd.merge(left = skr, right = skr_k4, how = 'left', on = '종목코드')
skr = pd.merge(left = skr, right = skr_k5, how = 'left', on = '종목코드')
skr = pd.merge(left = skr, right = skr_k6, how = 'left', on = '종목코드')
skr = pd.merge(left = skr, right = skr_k7, how = 'left', on = '종목코드')
skr = pd.merge(left = skr, right = skr_k8, how = 'left', on = '종목코드')
skr = pd.merge(left = skr, right = skr_k9, how = 'left', on = '종목코드')
skr = pd.merge(left = skr, right = skr_k10, how = 'left', on = '종목코드')
skr = pd.merge(left = skr, right = skr_k11, how = 'left', on = '종목코드')
skr = pd.merge(left = skr, right = skr_k12, how = 'left', on = '종목코드')
skr = pd.merge(left = skr, right = skr_k13, how = 'left', on = '종목코드')
skr = pd.merge(left = skr, right = skr_k14, how = 'left', on = '종목코드')
skr = pd.merge(left = skr, right = skr_k15, how = 'left', on = '종목코드')
skr = pd.merge(left = skr, right = skr_k16, how = 'left', on = '종목코드')
skr = skr.fillna(0)

C:\Users\user\AppData\Local\Temp\ipykernel_7000\1375084739.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skr_k5['사회기부/투자'] = 1
C:\Users\user\AppData\Local\Temp\ipykernel_7000\1375084739.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skr_k9['ISMS-P인증'] = 1
C:\Users\user\AppData\Local\Temp\ipykernel_7000\1375084739.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [28]:
### weight matrix
sics = pd.DataFrame({'소비재':[1/13,1/13,1/13,0,0,0,1/13,1/13,1/13,2/13,1/13,1/13,0,0,0,1/13,1/13,1/13],
                     '추출물 및 광물 처리':[1/11,1/11,1/11,0,0,0,0,0,0,0,1/11,1/11,2/11,2/11,2/11,0,0,0],
                     '금융':[1/9,1/9,1/9,0,0,0,1/9,1/9,1/9,0,0,0,0,0,0,1/9,1/9,1/9],
                     '식음료':[1/13,1/13,1/13,0,0,0,1/13,1/13,1/13,2/13,1/13,1/13,1/13,1/13,1/13,0,0,0],
                     '헬스케어':[1/17,1/17,1/17,1/17,1/17,1/17,2/17,2/17,2/17,2/17,0,0,0,0,0,1/17,1/17,1/17],
                     '인프라':[1/12,1/12,1/12,0,0,0,0,0,0,1/12,1/12,1/12,2/12,2/12,2/12,0,0,0],
                     '재생가능 자원 및 대체 에너지':[1/9,1/9,1/9,1/9,1/9,1/9,0,0,0,0,0,0,1/9,1/9,1/9,0,0,0],
                     '자원 변환':[1/14,1/14,1/14,1/14,1/14,1/14,1/14,1/14,1/14,2/14,0,0,1/14,1/14,1/14,0,0,0],
                     '서비스':[1/15,1/15,1/15,0,0,0,1/15,1/15,1/15,1/15,1/15,1/15,1/15,1/15,1/15,1/15,1/15,1/15],
                     '기술 및 통신':[1/20,1/20,1/20,0,0,0,2/20,2/20,2/20,0,1/20,1/20,1/20,1/20,1/20,2/20,2/20,2/20],
                     '운송':[1/12,1/12,1/12,0,0,0,0,0,0,1/12,1/12,1/12,2/12,2/12,2/12,0,0,0]})

In [29]:
### key risk : matrix 곱
a = [[0 for col in range(12)] for row in range(skr.shape[0])]
for i in range(skr.shape[0]) :
    a[i] = sics[skr['SICS_Sector_Kr'][i]].tolist()

skr['S_risk'] = np.diag(np.array(skr.iloc[:,2:]) @ np.array(a).T)

### Final Social Data

In [30]:
### modeling에 필요한 data split stratified 작업
social = pd.merge(left = social, right = skr[['종목코드','S_risk']], how = 'left', on = '종목코드')
social = social.fillna(0)

S_group = [0 for col in range(social.shape[0])]
for i in range(social.shape[0]) :
    if social['S_GRADE'].astype(int)[i] in [1,2,3] :
        S_group[i] = "G1"
    elif social['S_GRADE'].astype(int)[i] in [4,5,6] :
        S_group[i] = "G2"
    elif social['S_GRADE'].astype(int)[i] in [7,8,9] : 
        S_group[i] = "G3"
    else : S_group[i] = "NA"
        
social['Stratify'] = social['SICS_Sector_Kr'] + "_" + S_group
social[social['회사명'] == 'CJ ENM']['Stratify'] = '서비스_G2'  # 서비스 그룹에서 회사가 하나이므로 Group2로 변경
social.to_excel("C:/Users/user/Desktop/ESG데이터/S_final.xlsx",index = False)
social.to_excel("C:/Users/user/Desktop/ESG데이터/ESG/S_final.xlsx",index = False)
social.to_excel("C:/Users/user/Desktop/ESG데이터/ESGI/S_final.xlsx",index = False)
social.to_excel("C:/Users/user/Desktop/ESG데이터/Tableau/S_final.xlsx",index = False)

C:\Users\user\AppData\Local\Temp\ipykernel_7000\1535723906.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  social[social['회사명'] == 'CJ ENM']['Stratify'] = '서비스_G2'  # 서비스 그룹에서 회사가 하나이므로 Group2로 변경
